<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Schedulers

In the previous two notebooks we used `dask.delayed` and `dask.dataframe` to create computations.  By default, these ran in a local thread pool on our personal machines.  Often, this is sufficient, especially when you are bound by NumPy and Pandas routines which release the GIL and when you are using powerful workstation computers with many cores.

However sometimes you may want to execute your code in processes (for Pure Python code that holds onto the GIL), in a single thread (for profiling and debugging) or across a cluster (for larger computations).

In this section we first talk about changing schedulers.  Then we use the `dask.distributed` scheduler in more depth.

### Local Schedulers

Dask separates computation description (task graphs) from execution (schedulers). This allows you to write code once, and run it locally or scale it out across a cluster.

Here we discuss the *local* schedulers - schedulers that run only on a single machine. The three options here are:

- `dask.threaded.get         # uses a local thread pool`
- `dask.multiprocessing.get  # uses a local process pool`
- `dask.get                  # uses only the main thread (useful for debugging)`

In each case we change the scheduler used in a few different ways:

- By providing a `get=` keyword argument to `compute`:

```python
total.compute(get=dask.multiprocessing.get)
# or 
dask.compute(a, b, get=dask.multiprocessing.get)
```

- Using `dask.set_options`:

```python
# Use multiprocessing in this block
with dask.set_options(get=dask.multiprocessing.get):
    total.compute()
# Use multiprocessing globally
dask.set_options(get=dask.multiprocessing.get)
```

Here we repeat a simple dataframe computation from the previous section using the different schedulers:

In [ ]:
import dask 
import dask.multiprocessing
import dask.dataframe as dd
import pandas as pd
from glob import glob
import os

In [ ]:
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

# Maximum non-cancelled delay
largest_delay = df[~df.Cancelled].DepDelay.max()

In [ ]:
%time _ = largest_delay.compute()  # this uses threads by default

In [ ]:
%time _ = largest_delay.compute(get=dask.multiprocessing.get)  # this uses processes

In [ ]:
%time _ = largest_delay.compute(get=dask.get)  # This uses a single thread

By default the threaded and multiprocessing schedulers use the same number of workers as cores. You can change this using the `num_workers` keyword in the same way that you specified `get` above:

```
largest_delay.compute(get=dask.multiprocessing.get, num_workers=2)
```

To see how many cores you have on your computer, you can use `multiprocessing.cpu_count`

In [ ]:
from multiprocessing import cpu_count
cpu_count()

### Some Questions to Consider:

- How much speedup is possible for this task (hint, look at the graph).
- Given how many cores are on this machine, how much faster could the parallel schedulers be than the single-threaded scheduler.
- How much faster was using threads over a single thread? Why does this differ from the optimal speedup?
- Why is the multiprocessing scheduler so much slower here?

---

## In what cases would you want to use one scheduler over another?

http://dask.pydata.org/en/latest/scheduler-choice.html

---

## Profiling

*You should skip this section if you are running low on time*.

The synchronous scheduler is particularly valuable for debugging and profiling.  

For example, the IPython `%%prun` magic gives us profiling information about which functions take up the most time in our computation.  Try this magic on the computation above with different schedulers.  How informative is this magic when running parallel code?

In [ ]:
%prun _ = largest_delay.compute(get=dask.threaded.get)

In [ ]:
%prun _ = largest_delay.compute(get=dask.get)

To aid in profiling parallel execution, dask provides several [`diagnostics`](http://dask.pydata.org/en/latest/diagnostics.html) for measuring and visualizing performance. These are useful for seeing bottlenecks in the *parallel* computation, whereas the above `prun` is useful for seeing bottlenecks in individual *tasks*.

In [ ]:
from dask.diagnostics import Profiler, ResourceProfiler, visualize
from bokeh.io import output_notebook
output_notebook()

with Profiler() as p, ResourceProfiler(0.25) as r:
    largest_delay.compute()
    
visualize([r, p]);

From the plot above, we can see that while tasks are running concurrently, due to GIL effects we're only achieving parallelism during early parts of `pd.read_csv` (mostly the byte operations).


*It should be noted that the `dask.diagnostics` module is only useful when profiling on a single machine. The `dask.distributed` scheduler has its own set of diagnostics..*

---

## Distributed Scheduler

The `dask.distributed` system is composed of a single centralized scheduler and several worker processes.  We can either set these up manually as command line processes or have Dask set them up for us from the notebook.  


#### Automatically setup a local cluster

Starting a single scheduler and worker on the local machine is a common case. Dask will set up a local cluster for you if you provide no scheduler address to `Client`:

```python
from dask.distributed import Client
client = Client()
```

If you choose this approach then there is no need to set up a `dask-scheduler` or `dask-worker` process as described below.


#### Manually setup a distributed cluster

It is good to know how to set things up manually in case you want to try out Dask on a small cluster of your own.  However, if you are unfamiliar with the command line you can safely skip this section and go down to the Automatic section below. We run the `dask-scheduler` process on one machine.

```
$ dask-scheduler
distributed.scheduler - INFO -   Scheduler at:  tcp://127.0.0.1:8786
distributed.bokeh.application - INFO - Web UI: http://127.0.0.1:8787/status/
```

The scheduler reports that it is running at 127.0.0.1 on port 8786.  The address 127.0.0.1 is another name for "localhost" or "this machine".  We will need to give this address to the workers and client so you might want to copy it now.

We now run the `dask-worker` process on every machine that we want to use for computation.  For right now this is probably just once on our laptop, but in production this may be on many different machines:

```
$ dask-worker 127.0.0.1:8786
distributed.worker - INFO -       Start worker at:      tcp://127.0.0.1:45011
distributed.worker - INFO -         Registered to:      tcp://127.0.0.1:8786
```

*Note*: for this tutorial we want to start the dask-worker process in the `dask-tutorial-pydata-seattle/` directory.  This will ensure that the workers Python processes have access to the same data that our notebook process does.

```
$ cd dask-tutorial-pydata-seattle/  # navigate to whereever you have started your notebooks
~/dask-tutorial-pydata-seattle/ $ dask-worker 127.0.0.1:8786
```

*Note*: By default the dask-worker command line tool starts a single process with a thread pool with as many threads as you have cores on your computer.  If you are doing mostly GIL-released computations (numpy, pandas, scikit-learn) then this is the right choice.  However if you are doing mostly GIL-bound comptutations (Python code, pandas with text, parsing) then you will want to start the workers with multiple processes and one thread per process

```
$ dask-worker 127.0.0.1:8786 --nprocs 8 --nthreads 4
```
    
You can see more options by asking for help

```
$ dask-worker --help
```
    
When the scheduler and workers are running you can connect to them using a Dask `Client`, giving it the same address of the scheduler that you gave to the worker.

```python
from dask.distributed import Client
client = Client('127.0.0.1:8786')
```

### More information

You can find more information at the following documentation pages:

- [Quickstart](http://distributed.readthedocs.io/en/latest/quickstart.html)
- [Setup Network](http://distributed.readthedocs.io/en/latest/setup.html)

## Using a local cluster

As mentioned above, the multiprocessing scheduler can be inefficient for complicated workflows. The distributed scheduler doesn't have this downside, and works fine locally. This makes it often a good replacement for the multiprocessing scheduler, even when working on a single machine.

Here we startup a local cluster, and use it to repeat the same dataframe computation as done above:

In [ ]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client

In [ ]:
%time _ = largest_delay.compute(get=client.get)

#### Some Questions to Consider

- How does this compare to the optimal parallel speedup?
- Why is this faster than the threaded scheduler?

### Client takes over by default

Actually, we didn't need to add `get=client.get`.  The distributed scheduler takes over as the default scheduler for all collections when the Client is created:

In [ ]:
%time _ = largest_delay.compute()  # This used to use threads by default, now it uses dask.distributed

---

## Using a Distributed Cluster

We've setup a distributed cluster for you to use. The scheduler address is `schedulers:9000`. Here we create a new client to connect to it:

In [ ]:
client = Client('schedulers:9000')
client

#### Questions
- How many cores does this new cluster have?
- How many workers?

Here we run the same computation as above, but on the distributed cluster. As mentioned above, we don't need to specify the `get` function - the new `Client` took over as the default scheduler when it was created.

In [ ]:
%time _ = largest_delay.compute()  # This used to use threads by default, now it uses dask.distributed

### What Happened Here?

Why did this operation fail when it worked with the local cluster?

When working in a *distributed* environment you need to be careful about some things. When computing locally you can be sure that your workers share the same filesystem and python libraries. This is no longer true when working distributed.

In this case, the task graph contained references to a filepath that exists locally, but doesn't exist on the workers. A few common patterns when working with distributed data are:

- Store data in the cloud where it can be accessed by all workers (e.g. `s3`, `hdfs`, `gcfs`)
- Use a networed filesystem that's accessible to all nodes
- Duplicate the files on all nodes

In a future notebook we'll switch to using cloud storage. For now we'll change our example computation to use simulated flight data which doesn't rely on any files on disk. 

*Note: you don't need to understand this code.*

In [ ]:
# Create some fake flight data as a dask dataframe
# You don't need to understand this code

import numpy as np
import pandas as pd
from dask import delayed

@delayed(pure=True)
def make_chunk(start, end, N, seed):
    N = int(N)
    rs = np.random.RandomState(seed)
    origin = rs.choice(np.array(['EWR', 'LGA', 'JFK'], dtype='O'),
                       size=N, p=[0.45, 0.4, 0.15])    
    date = rs.randint(np.datetime64(start).astype('i8'),
                      np.datetime64(end).astype('i8'),
                      size=N).astype('M8[D]')
    cancelled = rs.random_sample(N) < 0.01
    date[cancelled] = None
    origin[cancelled] = None
    delay = np.full(N, np.NaN)
    for o, s in [('EWR', 0.4), ('LGA', 0.42), ('JFK', 0.47)]:
        m = origin == o
        delay[m] = (rs.gamma(s, 40, m.sum()) - 10).astype('i8')
    return pd.DataFrame({'Date': date,
                         'Origin': origin,
                         'DepDelay': delay,
                         'Cancelled': cancelled}).sort_values('Date')

chunks = [make_chunk('%d-01-01' % y, '%d-01-01' % (y + 1), 1e6, i)
          for i, y in enumerate(range(1990, 2000))]
df = dd.from_delayed(chunks)

In [ ]:
df

In [ ]:
largest_delay = df[~df.Cancelled].DepDelay.max()
largest_delay

In [ ]:
%time _ = largest_delay.compute()

---

## Diagnostics

One of the main advantages of using the distributed scheduler is the diagnostics dashboards that should be hosted [here](../9002/status).  Visit that link and then run the computation again.

You may want to arrange your notebook and this webpage side-by-side on your screen so that you can see both at the same time.

In [ ]:
%time _ = largest_delay.compute()

### Exercise

Run the following computations while looking at the diagnostics page. In each case what is taking the most time?

In [ ]:
# Number of flights
_ = len(df)

In [ ]:
# Number of non-cancelled flights
_ = len(df[~df.Cancelled])

In [ ]:
# Number of non-cancelled flights per-airport
_ = df[~df.Cancelled].groupby('Origin').Origin.count().compute()

In [ ]:
# Average departure delay from each airport?
_ = df[~df.Cancelled].groupby('Origin').DepDelay.mean().compute()

In [ ]:
# Average departure delay per day-of-week
_ = df.groupby(df.Date.dt.dayofweek).DepDelay.mean().compute()

---

### New API

The distributed scheduler is more sophisticated than the single machine schedulers.  It comes with more functions to manage data, computing in the background, and more.  The distributed scheduler also has entirely separate documentation

-  http://distributed.readthedocs.io/en/latest/
-  http://distributed.readthedocs.io/en/latest/api.html